## Solving Sudoku Puzzle

In [1]:
import numpy as np
import itertools

A sudoku is made of $3 \text{ blocks by } 3 \text{ blocks}$ and each block is a $3 \times 3$ squares.


$$\begin{array}{|c|c|c|}
\hline
\text{B1}&\text{B2}&\text{B3}\\
\hline
\text{B4}&\text{B5}&\text{B6}\\
\hline
\text{B7}&\text{B8}&\text{B9}\\
\hline
\end{array}$$


In a sudoku, each row has $9$ squares and each column has $9$ squares.


We requires to assign a number in $\{1,2,3,4,5,6,7,8,9\}$ to each square in the puzzle such that each row, each column and each block consist of an arrangements of $\{1,2,3,4,5,6,7,8,9\}$.

The function extract_a_board accepts a row name, a column name or a Block name and obtain its numbers on the puzzle.

In [2]:
#C - Column
#R - Row
#B - Block
def extract_a_board(a,type):
    if type[0]=='C':
        i = int(type[1])
        return a[:,i-1]
    elif type[0]=='R':
        i = int(type[1])
        return a[i-1,:]
    elif type[0]=='B':
        i = int(type[1])
        if i == 1:
            return a[0:3,0:3]
        elif i == 2:
            return a[0:3,3:6]
        elif i == 3:
            return a[0:3,6:9]
        elif i == 4:
            return a[3:6,0:3]
        elif i == 5:
            return a[3:6,3:6]
        elif i == 6:
            return a[3:6,6:9]
        elif i == 7:
            return a[6:9,0:3]
        elif i == 8:
            return a[6:9,3:6]
        elif i == 9:
            return a[6:9,6:9]
        else:
            return 'Error'
    else:
        return 'Error'

Given a position $(i,j)$, obtain its row, column and block symbol.

In [3]:
def locator(i,j):
    a = i//3
    b = j//3
    return f'R{i+1}',f'C{j+1}',f'B{3*a+b+1}'

Given a row name, a column name or a Block name, obtain its coordinates.

In [4]:
def extract_coors(type):
    if type[0]=='C':
        i = int(type[1])
        return [(x,i-1) for x in range(9)]
    elif type[0]=='R':
        i = int(type[1])
        return [(i-1,x) for x in range(9)]
    elif type[0]=='B':
        i = int(type[1])
        if i == 1:
            e = []
            for u in range(3):
                for v in range(3):
                    e.append((u,v))
            return e
        elif i == 2:
            e = []
            for u in range(3):
                for v in range(3,6):
                    e.append((u,v))
            return e
        elif i == 3:
            e = []
            for u in range(3):
                for v in range(6,9):
                    e.append((u,v))
            return e
        elif i == 4:
            e = []
            for u in range(3,6):
                for v in range(3):
                    e.append((u,v))
            return e
        elif i == 5:
            e = []
            for u in range(3,6):
                for v in range(3,6):
                    e.append((u,v))
            return e
        elif i == 6:
            e = []
            for u in range(3,6):
                for v in range(6,9):
                    e.append((u,v))
            return e
        elif i == 7:
            e = []
            for u in range(6,9):
                for v in range(3):
                    e.append((u,v))
            return e
        elif i == 8:
            e = []
            for u in range(6,9):
                for v in range(3,6):
                    e.append((u,v))
            return e
        elif i == 9:
            e = []
            for u in range(6,9):
                for v in range(6,9):
                    e.append((u,v))
            return e
        else:
            return 'Error'
    else:
        return 'Error'

For a given position $(i,j)$, we obtain its row, column and block on the puzzle. The numbers that are not on its row, column and block are available for arrangements.

In [5]:
def loc_available(i,j,a_board):
    a,b,c = locator(i,j)
    A = [1,2,3,4,5,6,7,8,9]
    U1 = [x for x in A if x not in extract_a_board(a_board,a).reshape(-1)]
    U2 = [x for x in A if x not in extract_a_board(a_board,b).reshape(-1)]
    U3 = [x for x in A if x not in extract_a_board(a_board,c).reshape(-1)]
    return np.intersect1d(np.intersect1d(U1,U2),U3)

For each integer $i$ in $\{1,2,3,4,5,6,7,8,9\}$, we find coordinates of rows, columns and blocks that contain $i$.

In [6]:
def covered_coors(i,abd):
    a,b = np.where(abd==i)
    pos = [(x,y) for x,y in zip(a,b)]
    cc = []
    for x,y in pos:
        u,v,z = locator(x,y)
        U = extract_coors(u)
        V = extract_coors(v)
        Z = extract_coors(z)
        cc.extend(list(set().union(*[U,V,Z])))
    cc = list(set(cc))
    cc.sort(key=lambda x: (x[0],x[1]))
    return cc

In [7]:
def uncovered_coors_in_block(type,i,bd):
    coors = covered_coors(i,bd)
    pcoors = [x for x in extract_coors(type) if x not in coors]
    pcoors = [x for x in pcoors if bd[x] == 0]
    return pcoors

In [8]:
def check_a_board(a_board):
    s = 0
    for i in range(9):
        if len(set(x for x in a_board[i,:] if x > 0)) < 9:
            s=1
    if s==0:
        for i in range(9):
            if len(set(x for x in a_board[:,i] if x > 0)) < 9:
                s=1
    if s==0:
        for i in range(1,10):
            t = f'B{i}'
            tc = extract_coors(t)
            if len(set(a_board[x] for x in tc if a_board[x] > 0)) < 9:
                s=1
    if s==0:
        return True
    else:
        return False

In [9]:
def solve(a_board):
    old_board = a_board.copy()
    stop = 0
    while stop==0:
        anum = {}
        new_board = old_board.copy()
        for i in range(9):
            for j in range(9):
                if new_board[i,j] == 0:
                    anum[(i,j)] = loc_available(i,j,new_board)
                else:
                    anum[(i,j)] = np.array([new_board[i,j]])
        for (i,j),k in anum.items():
            if len(k)==1:
                new_board[i,j] = k[0]
        if (new_board == old_board).all():
            stop = 1
        else:
            old_board = new_board.copy()
    stp = 0
    it = 0
    while stp==0:
        uncovered_coors = {}
        for i in range(1,10):
            ui = {}
            for k in range(1,10):
                ui[f'B{k}'] = uncovered_coors_in_block(f'B{k}',i,new_board)
            uncovered_coors[i] = ui
        for i in range(1,10):
            s = uncovered_coors[i]
            for k in range(1,10):
                if len(s[f'B{k}'])==1:
                    if new_board[s[f'B{k}'][0]] == 0:
                        new_board[s[f'B{k}'][0]] = i
        old_board = new_board.copy()
        stop = 0
        while stop==0:
            anum = {}
            new_board = old_board.copy()
            for i in range(9):
                for j in range(9):
                    if new_board[i,j] == 0:
                        anum[(i,j)] = loc_available(i,j,new_board)
                    else:
                        anum[(i,j)] = np.array([new_board[i,j]])
            for (i,j),k in anum.items():
                if len(k)==1:
                    if new_board[i,j] == 0:
                        new_board[i,j] = k[0]
            if (new_board == old_board).all():
                stop = 1
            else:
                old_board = new_board.copy()
        if (new_board > 0).all():
            stp=1
        elif it > 100:
            stp=1
        else:
            it = it+1
    return new_board

In [10]:
def find_match_block(abd):
    uncovered_coors = {}
    for i in range(1,10):
        ui = {}
        for k in range(1,10):
            ui[f'B{k}'] = uncovered_coors_in_block(f'B{k}',i,abd)
        uncovered_coors[i] = ui
    st = {}
    for r in range(2,5):
        st[r] = []
        comb2 = list(itertools.combinations([1,2,3,4,5,6,7,8,9], r))
        for u in comb2:
            s = []
            for x in list(uncovered_coors[u[0]].items()):
                p=0
                for j in range(1,len(u)):
                    if x not in list(uncovered_coors[u[j]].items()):
                        p=1
                if p==0:
                    if len(x[1])>0:
                        s.append((u,x))
            if len(s) > 0:
                for u,(x,y) in s:
                    if len(y)==r:
                        st[r].append((u,x,y))
    return st

In [11]:
def solve_sudoku(a_board):
    asboard = solve(a_board)
    if (asboard>0).all() and check_a_board(asboard):
        return asboard
    else:
        s = find_match_block(asboard)[2]
        cb = []
        for u,v,w in s:
            if len(cb)==0:
                cb.append([(u[0],w[0]),(u[1],w[1])])
                cb.append([(u[1],w[0]),(u[0],w[1])])
            else:
                cb2 = []
                for x in cb:
                    y = x.copy()
                    y.extend([(u[0],w[0]),(u[1],w[1])])
                    cb2.append(y)
                    y = x.copy()
                    y.extend([(u[1],w[0]),(u[0],w[1])])
                    cb2.append(y)
                cb = cb2.copy()
        stop = 0
        it = 0
        sp = 0
        while stop==0:
            if len(cb)>0:
                ait = cb[it]
                abd = asboard.copy()
                for u,v in ait:
                    abd[v] = u
                sabd = solve(abd)
                if (sabd > 0).all():
                    if check_a_board(sabd):
                        stop=1
                        sp=1
                it=it+1
                if it >= len(cb):
                    stop=1
            if sp==1:
                return sabd
            else:
                return 'Solution is not found.'
        else:
            return 'Solution is not found.'

In [12]:
a_board = np.array([[0,0,0,0,0,8,0,0,0],
                    [8,3,5,0,0,6,0,0,7],
                    [0,1,6,0,0,9,0,0,3],
                    [0,2,0,4,0,0,0,0,6],
                    [0,6,9,0,0,7,0,0,2],
                    [0,0,0,0,3,0,0,9,0],
                    [0,0,7,0,0,0,4,0,0],
                    [0,0,3,5,0,4,2,0,0],
                    [6,9,4,0,7,0,0,3,0]])

In [13]:
solve_sudoku(a_board)

array([[9, 7, 2, 3, 5, 8, 6, 1, 4],
       [8, 3, 5, 1, 4, 6, 9, 2, 7],
       [4, 1, 6, 7, 2, 9, 8, 5, 3],
       [3, 2, 1, 4, 9, 5, 7, 8, 6],
       [5, 6, 9, 8, 1, 7, 3, 4, 2],
       [7, 4, 8, 6, 3, 2, 1, 9, 5],
       [2, 5, 7, 9, 8, 3, 4, 6, 1],
       [1, 8, 3, 5, 6, 4, 2, 7, 9],
       [6, 9, 4, 2, 7, 1, 5, 3, 8]])

In [14]:
a_board = np.array([[7,0,0,1,0,0,0,0,9],
                    [0,0,6,0,0,0,3,8,4],
                    [9,5,4,0,0,6,1,0,0],
                    [1,3,0,0,0,0,0,0,0],
                    [0,0,2,7,0,0,0,0,0],
                    [0,0,0,0,0,0,9,7,0],
                    [0,0,0,0,0,1,8,6,0],
                    [8,7,0,0,6,9,0,3,0],
                    [0,0,1,3,0,8,0,0,2]])

In [15]:
solve_sudoku(a_board)

array([[7, 8, 3, 1, 2, 4, 6, 5, 9],
       [2, 1, 6, 5, 9, 7, 3, 8, 4],
       [9, 5, 4, 8, 3, 6, 1, 2, 7],
       [1, 3, 7, 9, 8, 5, 2, 4, 6],
       [6, 9, 2, 7, 4, 3, 5, 1, 8],
       [5, 4, 8, 6, 1, 2, 9, 7, 3],
       [3, 2, 9, 4, 7, 1, 8, 6, 5],
       [8, 7, 5, 2, 6, 9, 4, 3, 1],
       [4, 6, 1, 3, 5, 8, 7, 9, 2]])

In [16]:
a_board = np.array([[0,0,0,1,0,0,3,0,0],
                    [0,0,3,2,0,7,1,9,5],
                    [0,0,9,0,0,8,0,7,0],
                    [0,0,0,5,0,0,2,0,1],
                    [0,0,0,0,0,0,0,5,0],
                    [0,0,0,3,1,2,0,0,8],
                    [0,5,0,0,0,0,0,0,0],
                    [0,0,0,9,4,0,0,0,0],
                    [0,4,0,7,8,3,0,0,9]])

In [17]:
solve_sudoku(a_board)

'Solution is not found.'

In [18]:
a_board = np.array([[0,0,0,6,0,0,0,0,5],
                    [0,0,0,0,5,0,6,8,4],
                    [0,0,0,0,0,3,0,0,0],
                    [0,2,3,0,0,4,0,6,7],
                    [0,8,6,0,0,0,3,2,0],
                    [0,5,0,3,0,0,4,0,0],
                    [0,0,0,0,0,0,7,9,2],
                    [0,3,7,0,2,0,8,5,0],
                    [0,0,0,0,0,0,0,0,0]])

In [19]:
solve_sudoku(a_board)

'Solution is not found.'

In [20]:
a_board = np.array([[0,3,1,5,0,0,0,0,0],
                    [0,7,0,0,9,0,2,0,0],
                    [9,0,5,0,0,0,6,0,0],
                    [7,0,0,2,0,0,0,0,9],
                    [0,4,9,0,0,0,0,0,0],
                    [3,0,0,0,0,4,5,0,0],
                    [0,1,0,0,0,0,4,9,0],
                    [0,9,4,6,0,5,7,0,0],
                    [6,0,0,0,0,0,0,0,3]])

In [21]:
solve_sudoku(a_board)

array([[2, 3, 1, 5, 7, 6, 9, 8, 4],
       [4, 7, 6, 3, 9, 8, 2, 1, 5],
       [9, 8, 5, 4, 2, 1, 6, 3, 7],
       [7, 5, 8, 2, 6, 3, 1, 4, 9],
       [1, 4, 9, 8, 5, 7, 3, 6, 2],
       [3, 6, 2, 9, 1, 4, 5, 7, 8],
       [5, 1, 3, 7, 8, 2, 4, 9, 6],
       [8, 9, 4, 6, 3, 5, 7, 2, 1],
       [6, 2, 7, 1, 4, 9, 8, 5, 3]])

In [22]:
a_board = np.array([[0,0,0,7,0,0,2,0,8],
                    [3,0,0,0,0,0,5,0,0],
                    [9,5,0,0,0,0,0,0,0],
                    [0,0,0,0,0,3,9,7,0],
                    [0,9,0,4,0,7,0,5,0],
                    [4,0,0,6,0,0,0,0,3],
                    [7,8,0,0,1,0,4,0,5],
                    [5,0,0,0,9,0,8,0,7],
                    [0,1,0,0,0,0,0,0,0]])

In [23]:
solve_sudoku(a_board)

array([[6, 4, 1, 7, 3, 5, 2, 9, 8],
       [3, 7, 8, 9, 6, 2, 5, 4, 1],
       [9, 5, 2, 8, 4, 1, 7, 3, 6],
       [8, 6, 5, 1, 2, 3, 9, 7, 4],
       [1, 9, 3, 4, 8, 7, 6, 5, 2],
       [4, 2, 7, 6, 5, 9, 1, 8, 3],
       [7, 8, 9, 3, 1, 6, 4, 2, 5],
       [5, 3, 6, 2, 9, 4, 8, 1, 7],
       [2, 1, 4, 5, 7, 8, 3, 6, 9]])

In [24]:
a_board = np.array([[0,0,5,8,7,1,0,4,0],
                    [0,0,7,9,6,0,0,0,0],
                    [0,0,0,0,0,5,0,0,0],
                    [4,0,0,2,5,0,6,0,0],
                    [0,8,6,0,9,0,1,0,0],
                    [0,0,0,0,0,0,2,9,0],
                    [5,0,0,0,0,0,0,0,0],
                    [0,0,0,0,0,6,0,5,1],
                    [0,6,8,0,2,0,7,0,0]])

In [25]:
solve_sudoku(a_board)

array([[6, 9, 5, 8, 7, 1, 3, 4, 2],
       [3, 4, 7, 9, 6, 2, 5, 1, 8],
       [8, 2, 1, 3, 4, 5, 9, 6, 7],
       [4, 1, 9, 2, 5, 7, 6, 8, 3],
       [2, 8, 6, 4, 9, 3, 1, 7, 5],
       [7, 5, 3, 6, 1, 8, 2, 9, 4],
       [5, 7, 4, 1, 3, 9, 8, 2, 6],
       [9, 3, 2, 7, 8, 6, 4, 5, 1],
       [1, 6, 8, 5, 2, 4, 7, 3, 9]])

In [26]:
a_board = np.array([[2., 3., 5., 0., 6., 0., 0., 0., 4.],
                    [1., 6., 8., 0., 0., 0., 0., 0., 0.],
                    [4., 7., 9., 3., 0., 0., 8., 0., 0.],
                    [0., 0., 2., 6., 1., 3., 0., 9., 0.],
                    [0., 1., 0., 2., 4., 8., 0., 0., 6.],
                    [3., 0., 0., 5., 7., 9., 4., 0., 0.],
                    [7., 0., 0., 0., 3., 0., 6., 5., 9.],
                    [0., 0., 0., 9., 0., 0., 3., 4., 7.],
                    [0., 5., 0., 0., 0., 0., 2., 8., 1.]])

In [27]:
solve_sudoku(a_board)

'Solution is not found.'

### Generating a new solvable Sudoku Puzzle

In [28]:
def fs():
    SP = np.zeros((9,9))
    B1 = (np.random.choice(9,size=9,replace=False)+1).reshape((3,3))
    SP[:3,:3] = B1
    B5 = (np.random.choice(9,size=9,replace=False)+1).reshape((3,3))
    SP[3:6,3:6] = B5
    B9 = (np.random.choice(9,size=9,replace=False)+1).reshape((3,3))
    SP[6:9,6:9] = B9
    return SP

In [33]:
def gen_a_board(nr_of_visibles):
    stp = 0
    while stp==0:
        print(stp)
        combs = []
        for i in range(9):
            for j in range(9):
                combs.append((i,j))
        A = [1,2,3,4,5,6,7,8,9]
        #Generate a board with diagonal blocks entries
        #entries in each diagonal block are random digits from A
        #rdict contains the list of allowable digits for every positions
        SP = fs()
        rdict = {}
        for i,j in combs:
            if SP[i,j] > 0:
                rdict[(i,j)] = [SP[i,j]]
            else:
                r,c,b = locator(i,j)
                Sr = extract_coors(r)
                Sc = extract_coors(c)
                Sb = extract_coors(b)
                Br = [SP[u,v] for u,v in Sr if SP[u,v]>0]
                Bc = [SP[u,v] for u,v in Sc if SP[u,v]>0]
                Bb = [SP[u,v] for u,v in Sb if SP[u,v]>0]
                K = [x for x in A if x not in np.union1d(np.union1d(Br,Bc),Bb)]
                rdict[(i,j)] = K
        #IM is a sub-dict of rdict which contains only those positions
        #where less than or equal to 3 digits are possible.
        IM = {}
        for i,j in combs:
            if SP[i,j] == 0:
                if len(rdict[(i,j)]) <= 3:
                    IM[(i,j)] = rdict[(i,j)]
        #possible_assignments contains all the possible arrangements of possible digits (stored in IM)
        #to the positions
        possible_assignments = []
        for l in itertools.product(*list(IM.values())):
            possible_assignments.append(l)
        #for each possible assignment, solve the board and output the first solvable board
        if len(possible_assignments) < 100000:
            stop = 0
            l = 0
            while stop == 0:
                SP2 = SP.copy()
                k = 0
                for i,j in IM:
                    SP2[i,j] = possible_assignments[l][k]
                    k = k+1
                ans = solve_sudoku(SP2)
                if isinstance(ans,str)==False:
                    if check_a_board(ans):
                        stop = 1
                l = l+1
                if l >= len(possible_assignments):
                    stop = 1
            print(l)
            if l < len(possible_assignments):
                nr_of_non_visibles = 81 - nr_of_visibles
                pos_to_zeros = [combs[x] for x in np.random.choice(range(len(combs)),
                                                                   size=nr_of_non_visibles,replace=False)]
                for i,j in pos_to_zeros:
                    ans[i,j] = 0
                stp=1
                #print(stp)
    return ans

In [34]:
gen_a_board(27)

0
0
0
0
2480


array([[1., 5., 0., 6., 0., 7., 9., 3., 0.],
       [9., 0., 0., 0., 0., 8., 0., 4., 5.],
       [0., 0., 0., 0., 0., 5., 8., 0., 0.],
       [0., 0., 0., 0., 8., 6., 0., 2., 0.],
       [0., 9., 2., 5., 7., 0., 3., 0., 6.],
       [6., 0., 0., 0., 9., 3., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 3., 0., 0., 0., 0., 0., 0., 0.],
       [0., 6., 0., 0., 0., 0., 2., 0., 0.]])